<a href="https://colab.research.google.com/github/kiwihero/icecream-recommendation-system/blob/simpleRNN/ice_cream_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing import sequence 
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Embedding,Flatten, Dot, Input,Dropout,LSTM,Concatenate
from keras import initializers
from keras.preprocessing.text import Tokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# processing data set and filtering out columns that have no information
products = pd.read_csv("/content/drive/MyDrive/Kaggle/combined/products.csv")
products = products.drop(["subhead"],1)

reviews = pd.read_csv("/content/drive/MyDrive/Kaggle/combined/reviews.csv")
reviews=reviews.drop(["ingredients","texture", "likes","taste"],axis=1)



In [ ]:
train, test = train_test_split(products, test_size=0.2, random_state=42)
p_key = len(products.key.unique())
p_name = len(products.name.unique())

In [ ]:

product_input = Input(shape=[1], name="Produc-Input")
product_embedding = Embedding(p_name+1, 5, name="Product-Embedding")(product_input)
product_vec = Flatten(name="Flatten-Products")(product_embedding)

key_input = Input(shape=[1], name="Key-Input")
key_embedding = Embedding(p_key+1, 5, name="key-Embedding")(key_input)
key_vec = Flatten(name="Flatten-Keys")(key_embedding)

# building RNN model
max_features = 3000
rnn_hidden_dim = 100
word_embedding_dim = 30000
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))
model_rnn.add(Dropout(0.4))
model_rnn.add(LSTM(32))
model_rnn.add(Dropout(0.4))
model_rnn.add(Dense(237, activation='sigmoid'))

rmsprop = keras.optimizers.RMSprop(lr = .1)

model_rnn.compile(loss='SparseCategoricalCrossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 30000)       90000000  
_________________________________________________________________
dropout (Dropout)            (None, None, 30000)       0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                3844224   
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 237)               7821      
Total params: 93,852,045
Trainable params: 93,852,045
Non-trainable params: 0
_________________________________________________________________


In [ ]:
batch_size = 128
history = model_rnn.fit([train.key, train.name], train.rating,
          batch_size=batch_size,
          epochs=10,
          verbose = 1)

Epoch 1/10


UnimplementedError: ignored

In [ ]:
if os.path.exists('regression_model2.h5'):
    model2 = load_model('regression_model2.h5')
else:
    history = model2.fit([train.key, train.name], train.rating, epochs=10, verbose=1)
    model2.save('regression_model2.h5')
    plt.plot(history.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")

## We will use Keras evaluate function to evaluate performance on the test set

score = model_rnn.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


# Write a function to plot the train & test losses
def plot_loss_accuracy(history):
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 2, 1)
    ax.plot(history.history["loss"],'r-x', label="Train Loss")
    ax.plot(history.history["val_loss"],'b-x', label="Validation Loss")
    ax.legend()
    ax.set_title('cross_entropy loss')
    ax.grid(True)


    ax = fig.add_subplot(1, 2, 2)
    ax.plot(history.history["accuracy"],'r-x', label="Train Accuracy")
    ax.plot(history.history["val_accuracy"],'b-x', label="Validation Accuracy")
    ax.legend()
    ax.set_title('accuracy')
    ax.grid(True)  

# Calling the function
plot_loss_accuracy(history)


Epoch 1/10


UnimplementedError: ignored